<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

In [50]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [51]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Freeze Base Model

In [52]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [53]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [54]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

In [55]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[keras.metrics.BinaryAccuracy()])

## Augment the Data

In [56]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
transformation_ratio = 0.05
datagen = ImageDataGenerator(rescale=1. / 255,
                             validation_split = 0.2,
                             rotation_range=transformation_ratio,
                             shear_range=transformation_ratio,
                             zoom_range=transformation_ratio,
                             cval=transformation_ratio,
                             horizontal_flip=True,
                             vertical_flip=True)


## Load Dataset

In [57]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

In [58]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=16)

Epoch 1/16
37/36 [==============================] - 19s 501ms/step - loss: 0.7465 - binary_accuracy: 0.8376 - val_loss: 0.7406 - val_binary_accuracy: 0.8399
Epoch 2/16
37/36 [==============================] - 18s 499ms/step - loss: 0.7371 - binary_accuracy: 0.8481 - val_loss: 0.7311 - val_binary_accuracy: 0.8566
Epoch 3/16
37/36 [==============================] - 19s 502ms/step - loss: 0.7293 - binary_accuracy: 0.8589 - val_loss: 0.7247 - val_binary_accuracy: 0.8708
Epoch 4/16
37/36 [==============================] - 18s 496ms/step - loss: 0.7221 - binary_accuracy: 0.8766 - val_loss: 0.7180 - val_binary_accuracy: 0.8779
Epoch 5/16
37/36 [==============================] - 19s 501ms/step - loss: 0.7166 - binary_accuracy: 0.8837 - val_loss: 0.7113 - val_binary_accuracy: 0.8911
Epoch 6/16
37/36 [==============================] - 18s 498ms/step - loss: 0.7108 - binary_accuracy: 0.8921 - val_loss: 0.7065 - val_binary_accuracy: 0.8982
Epoch 7/16
37/36 [==============================] - 18s 49

## Unfreeze Model for Fine Tuning

In [59]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [61]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=12)

Epoch 1/12
37/36 [==============================] - 20s 533ms/step - loss: 0.6325 - binary_accuracy: 0.9955 - val_loss: 0.6337 - val_binary_accuracy: 0.9939
Epoch 2/12
37/36 [==============================] - 20s 534ms/step - loss: 0.6320 - binary_accuracy: 0.9966 - val_loss: 0.6338 - val_binary_accuracy: 0.9919
Epoch 3/12
37/36 [==============================] - 20s 529ms/step - loss: 0.6318 - binary_accuracy: 0.9966 - val_loss: 0.6339 - val_binary_accuracy: 0.9939
Epoch 4/12
37/36 [==============================] - 20s 545ms/step - loss: 0.6315 - binary_accuracy: 0.9969 - val_loss: 0.6332 - val_binary_accuracy: 0.9949
Epoch 5/12
37/36 [==============================] - 20s 527ms/step - loss: 0.6308 - binary_accuracy: 0.9983 - val_loss: 0.6335 - val_binary_accuracy: 0.9929
Epoch 6/12
37/36 [==============================] - 20s 531ms/step - loss: 0.6317 - binary_accuracy: 0.9968 - val_loss: 0.6330 - val_binary_accuracy: 0.9949
Epoch 7/12
37/36 [==============================] - 20s 54

In [62]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 323ms/step - loss: 0.6368 - binary_accuracy: 0.9868


[0.6368132829666138, 0.9868288636207581]

## Run the Assessment

In [63]:
from run_assessment import run_assessment

In [64]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 336ms/step - loss: 0.6360 - binary_accuracy: 0.9899

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9888.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.
